In [1]:
from sympy import *
init_printing()

## Characteristic constants

In [2]:
q = 19
d = 3

In [3]:
c = [ Matrix(x) for x in [
    ( 0, 1, 1), (-1, 0, 1), ( 0, 0, 1), ( 1, 0, 1), ( 0, -1, 1),
    (-1, 1, 0), ( 0, 1, 0), ( 1, 1, 0), (-1, 0, 0), ( 0, 0, 0), ( 1, 0, 0), (-1,-1, 0), ( 0, -1, 0), ( 1, -1, 0),
    ( 0, 1,-1), (-1, 0,-1), ( 0, 0,-1), ( 1, 0,-1), ( 0, -1,-1)
]]

In [4]:
c

⎡⎡0⎤  ⎡-1⎤  ⎡0⎤  ⎡1⎤  ⎡0 ⎤  ⎡-1⎤  ⎡0⎤  ⎡1⎤  ⎡-1⎤  ⎡0⎤  ⎡1⎤  ⎡-1⎤  ⎡0 ⎤  ⎡1 ⎤  
⎢⎢ ⎥  ⎢  ⎥  ⎢ ⎥  ⎢ ⎥  ⎢  ⎥  ⎢  ⎥  ⎢ ⎥  ⎢ ⎥  ⎢  ⎥  ⎢ ⎥  ⎢ ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥  
⎢⎢1⎥, ⎢0 ⎥, ⎢0⎥, ⎢0⎥, ⎢-1⎥, ⎢1 ⎥, ⎢1⎥, ⎢1⎥, ⎢0 ⎥, ⎢0⎥, ⎢0⎥, ⎢-1⎥, ⎢-1⎥, ⎢-1⎥, 
⎢⎢ ⎥  ⎢  ⎥  ⎢ ⎥  ⎢ ⎥  ⎢  ⎥  ⎢  ⎥  ⎢ ⎥  ⎢ ⎥  ⎢  ⎥  ⎢ ⎥  ⎢ ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥  
⎣⎣1⎦  ⎣1 ⎦  ⎣1⎦  ⎣1⎦  ⎣1 ⎦  ⎣0 ⎦  ⎣0⎦  ⎣0⎦  ⎣0 ⎦  ⎣0⎦  ⎣0⎦  ⎣0 ⎦  ⎣0 ⎦  ⎣0 ⎦  

⎡0 ⎤  ⎡-1⎤  ⎡0 ⎤  ⎡1 ⎤  ⎡0 ⎤⎤
⎢  ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥⎥
⎢1 ⎥, ⎢0 ⎥, ⎢0 ⎥, ⎢0 ⎥, ⎢-1⎥⎥
⎢  ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥⎥
⎣-1⎦  ⎣-1⎦  ⎣-1⎦  ⎣-1⎦  ⎣-1⎦⎦

In [5]:
w = [Rational(*x) for x in [
    (1,36), (1,36), (1,18), (1,36), (1,36),
    (1,36), (1,18), (1,36), (1,18), (1,3), (1,18), (1,36), (1,18), (1,36),
    (1,36), (1,36), (1,18), (1,36), (1,36)
]]

In [6]:
w

In [7]:
sum(w)

In [8]:
c_s = sqrt(Rational(1,3))
c_s

## Moments

In [9]:
rho, tau = symbols('rho tau')

In [10]:
f_next = symarray('f_next', q)
f_next

array([f_next_0, f_next_1, f_next_2, f_next_3, f_next_4, f_next_5,
       f_next_6, f_next_7, f_next_8, f_next_9, f_next_10, f_next_11,
       f_next_12, f_next_13, f_next_14, f_next_15, f_next_16, f_next_17,
       f_next_18], dtype=object)

In [11]:
f_curr = symarray('f_curr', q)
f_curr

array([f_curr_0, f_curr_1, f_curr_2, f_curr_3, f_curr_4, f_curr_5,
       f_curr_6, f_curr_7, f_curr_8, f_curr_9, f_curr_10, f_curr_11,
       f_curr_12, f_curr_13, f_curr_14, f_curr_15, f_curr_16, f_curr_17,
       f_curr_18], dtype=object)

In [12]:
u = Matrix(symarray('u', d))
u

⎡u₀⎤
⎢  ⎥
⎢u₁⎥
⎢  ⎥
⎣u₂⎦

In [13]:
from sympy.codegen.ast import Assignment

In [14]:
moments = [ Assignment(rho, sum(f_curr)) ]
moments

In [15]:
for i, u_i in enumerate(u):
    moments.append(Assignment(u_i, sum([ (c_j*f_curr[j])[i] for j, c_j in enumerate(c) ]) / sum(f_curr)))

moments

In [16]:
count_ops(moments, visual=True)

In [17]:
moments_opt = cse(moments, optimizations='basic', symbols=numbered_symbols(prefix='m'))
moments_opt

In [18]:
count_ops(moments_opt, visual=True)

In [19]:
for expr in moments_opt[0]:
    print("const float %s = %s;" % (expr[0], ccode(expr[1])))

const float m0 = f_curr_11 + f_curr_15 + f_curr_8;
const float m1 = f_curr_14 + f_curr_6 + f_curr_7;
const float m2 = f_curr_2 + f_curr_3 + f_curr_4;
const float m3 = f_curr_0 + f_curr_1 + f_curr_10 + f_curr_12 + f_curr_13 + f_curr_16 + f_curr_17 + f_curr_18 + f_curr_5 + f_curr_9 + m0 + m1 + m2;
const float m4 = -f_curr_13 + f_curr_5;
const float m5 = f_curr_1 - f_curr_17;
const float m6 = 1.0/m3;
const float m7 = f_curr_0 - f_curr_18;


In [20]:
for i, expr in enumerate(moments_opt[1]):
    print(ccode(expr))

rho = m3;
u_0 = -m6*(-f_curr_10 - f_curr_3 - f_curr_7 + m0 + m4 + m5);
u_1 = m6*(-f_curr_11 - f_curr_12 - f_curr_4 + m1 + m4 + m7);
u_2 = m6*(-f_curr_14 - f_curr_15 - f_curr_16 + m2 + m5 + m7);


## Equilibrium

In [21]:
f_eq = []

for i, c_i in enumerate(c):
    f_eq_i = w[i] * rho * (  1
                           + c_i.dot(u)    /    c_s**2
                           + c_i.dot(u)**2 / (2*c_s**4)
                           - u.dot(u)      / (2*c_s**2) )
    f_eq.append(f_eq_i)

f_eq

## Collision

In [22]:
collide = [ Assignment(f_next[i], f_curr[i] + 1/tau * ( f_eq_i - f_curr[i] )) for i, f_eq_i in enumerate(f_eq) ]
collide

In [23]:
count_ops(collide, visual=True)

In [24]:
collide_opt = cse(collide, optimizations='basic')
collide_opt

In [25]:
count_ops(collide_opt, visual=True)

In [26]:
for expr in collide_opt[0]:
    print("const float %s = %s;" % (expr[0], ccode(expr[1])))

const float x0 = 9*pow(u_1 + u_2, 2);
const float x1 = 6*u_2;
const float x2 = pow(u_1, 2);
const float x3 = 3*x2;
const float x4 = -x3;
const float x5 = pow(u_0, 2);
const float x6 = 3*x5;
const float x7 = -x6 + 2;
const float x8 = x4 + x7;
const float x9 = x1 + x8;
const float x10 = pow(u_2, 2);
const float x11 = 3*x10;
const float x12 = -x11;
const float x13 = 6*u_1;
const float x14 = x12 + x13;
const float x15 = 1.0/tau;
const float x16 = (1.0/72.0)*x15;
const float x17 = 6*u_0;
const float x18 = -u_0;
const float x19 = x11 + x3 - 2;
const float x20 = x19 + x6;
const float x21 = -x1 + x20;
const float x22 = 6*x10;
const float x23 = (1.0/36.0)*x15;
const float x24 = 9*pow(u_0 + u_2, 2);
const float x25 = x12 + x17;
const float x26 = -u_1;
const float x27 = -x13;
const float x28 = x17 + x20;
const float x29 = 6*x2;
const float x30 = 9*pow(u_0 + u_1, 2);
const float x31 = 6*x5;
const float x32 = x6 - 2;
const float x33 = -x17 + x20;
const float x34 = -u_2;
const float x35 = x1 + x20;


In [27]:
for i, expr in enumerate(collide_opt[1]):
    print(ccode(expr))

f_next_0 = f_curr_0 - x16*(72*f_curr_0 - rho*(x0 + x14 + x9));
f_next_1 = f_curr_1 - x16*(72*f_curr_1 + rho*(x17 + x21 - 9*pow(u_2 + x18, 2)));
f_next_2 = f_curr_2 - x23*(36*f_curr_2 - rho*(x22 + x9));
f_next_3 = f_curr_3 - x16*(72*f_curr_3 - rho*(x24 + x25 + x9));
f_next_4 = f_curr_4 - x16*(72*f_curr_4 + rho*(x13 + x21 - 9*pow(u_2 + x26, 2)));
f_next_5 = f_curr_5 - x16*(72*f_curr_5 + rho*(x27 + x28 - 9*pow(u_1 + x18, 2)));
f_next_6 = f_curr_6 - x23*(36*f_curr_6 - rho*(x14 + x29 + x7));
f_next_7 = f_curr_7 - x16*(72*f_curr_7 - rho*(x14 + x17 + x30 + x8));
f_next_8 = f_curr_8 - x23*(36*f_curr_8 + rho*(x17 + x19 - x31));
f_next_9 = f_curr_9 - 1.0/6.0*x15*(6*f_curr_9 + rho*x20);
f_next_10 = f_curr_10 - x23*(36*f_curr_10 - rho*(x25 + x31 + x4 + 2));
f_next_11 = f_curr_11 - x16*(72*f_curr_11 + rho*(x13 + x28 - x30));
f_next_12 = f_curr_12 - x23*(36*f_curr_12 + rho*(x11 + x13 - x29 + x32));
f_next_13 = f_curr_13 - x16*(72*f_curr_13 + rho*(x13 + x33 - 9*pow(u_0 + x26, 2)));
f_next_14 = f_curr